<a href="https://colab.research.google.com/github/gabelstein/2019_elegANT/blob/master/emgRiemannFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip show pyriemann

In [2]:
!pip install -q pyriemann
!pip install -q thundersvm
!wget -q https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update -q
!apt-get install cuda=9.0.176-1 -V -q


     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 512kB 2.7MB/s 
Selecting previously unselected package cuda-repo-ubuntu1704-9-0-local.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...
OK
Get:1 file:/var/cuda-repo-9-0-local  InRelease
Ign:1 file:/var/cuda-repo-9-0-local  InRelease
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRel

In [0]:
import numpy as np
import pyriemann as pr
import scipy as sp
import scipy.io as sio
import sys
import time
import thundersvm as tsvm
import json
import datetime


from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

from sklearn.model_selection import cross_val_score

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [0]:
def load_data(pname):
    '''
    Gets a participants code and return the data.

    Parameters
    ----------
    pname: str
        code for participant (e.g. S010)

    Returns
    ----------
    tuple of dictionaries
        dictionaries with full data and auxillary data

    '''
    full_data = sio.loadmat(f"/content/drive/My Drive/emgData/S{pname}_comp_short.mat")
    grasp = full_data["grasp"].flatten()
    grasp_rep = full_data["grasprepetition"].flatten()
    emg = full_data["emg"]
    return emg, grasp, grasp_rep


def save_data(pname, dictionary):
    '''
    Gets a participants code and dictionary and saves the data.

    Parameters
    ----------
    pname: str
        code for participant (e.g. S010)
    dictionary: dict
        dictionary for data

    '''
    sio.savemat(f"/Volumes/Steam Library/LR_Data/Dataset 2/{pname}_short.mat", dictionary)


def classify_MDM(X, y):
    '''
    Classifies a given data-set using the Riemannian manifold classifier.

    Parameters
    ----------
    X: ndarray
        EEG data, in format Ntrials x Nchannels X Nsamples
    y: ndarray
        corresponding labels for X

    Returns
    ----------
    float
        accuracy for the classification (5-fold crossvalidation)
    '''
    startcov = time.time()
    cov = pr.estimation.Covariances().fit_transform(X)
    endcov = time.time()
    print(f"estimated covariances in {(endcov - startcov):.2f} seconds.")
    startcov = time.time()

    mdm = pr.classification.MDM(n_jobs=2)
    accuracy = cross_val_score(mdm, cov, y)
    endcov = time.time()
    print(f"classify with MDM in {(endcov - startcov):.2f} seconds.")

    return accuracy

def classify(X, y, classifier_params):
    '''
    Classifies a given data-set on Riemannian manifold.

    Parameters
    ----------
    X: ndarray
        EEG data, in format Ntrials x Nchannels X Nsamples
    y: ndarray
        corresponding labels for X

    Returns
    ----------
    float
        accuracy for the classification (5-fold crossvalidation)
    '''

    classifier = classifier_params["classifier"]
    params = classifier_params["params"]

    startcov = time.time()
  
    accuracy = cross_val_score(classifier(**params), X, y, n_jobs=2).mean()
    
    endcov = time.time()
    print(f"classify with {classifier.__name__} in {(endcov - startcov):.2f} seconds. result: {accuracy:.2f}")

    return accuracy

'''
def classify_on_tangentspace(cov_ts, y, classifier, classifier_params):
    """
    Classifies a given data-set using the Riemannian manifold classifier.

    Parameters
    ----------
    X: ndarray
        EEG data, in format Ntrials x Nchannels X Nsamples
    y: ndarray
        corresponding labels for X

    Returns
    ----------
    float
        accuracy for the classification (5-fold crossvalidation)
    """

    startcov = time.time()
    accuracy = cross_val_score(classifier(**classifier_params), cov_ts, y, n_jobs=2).mean()
    endcov = time.time()
    print(f"classify with {classifier.__name__} in {(endcov - startcov):.2f} seconds. result: {accuracy:.2f}")

    return accuracy
'''

def make_sliding_subarrays(array, window_size, step_size):
    length = array.shape[0]
    steps = int(np.ceil(window_size / step_size))

    result = [[] for i in range(steps)]
    for i in range(0, steps):
        split = np.split(array, range(i * step_size, length, window_size), axis=0)
        if len(split) > 1:
            if split[0].shape[0] < window_size:
                split = split[1:]
            if split[-1].shape[0] < window_size:
                split = split[:-1]

            result[i] = np.array(split).swapaxes(1, 2)
        else:
            print(split)
            print(array)
    return np.vstack(result)


def sliding_windows_X(emg, grasp, grasprepetition, intlength=1000, step_size=100):
    valid_reps = grasprepetition >= 0
    grasps = np.unique(grasp)
    n_grasps = grasps.size
    X = [[] for i in range(n_grasps)]
    y = [[] for i in range(n_grasps)]

    for g in grasps:
        true_array = (valid_reps * (grasp == g))

        data = emg[true_array, :]
        X[g - 1] = make_sliding_subarrays(data, intlength, step_size)
        y[g - 1] = np.ones(X[g - 1].shape[0]) * g
    return np.concatenate(X), np.concatenate(y)

def make_Xy(emg,grasp,grasp_rep,intlength=400,step_size=200):
    start = time.time()
    X, y = sliding_windows_X(emg, grasp, grasp_rep, intlength=intlength, step_size=step_size)
    end = time.time()
    print(f"X,y sliding window time: {(end - start):.2f}s for {X.shape}")
    return X,y
  
def make_cov(X):
    startcov = time.time()
    cov = pr.estimation.Covariances().fit_transform(X)
    endcov = time.time()
    print(f"covariance estimation in {(endcov - startcov):.2f} seconds.")
    return cov
  
def ts_projection(cov):
    startcov = time.time()
    ts = TangentSpace()
    cov_ts = ts.fit_transform(cov)
    endcov = time.time()
    print(f"tangent space projection in {(endcov - startcov):.2f} seconds.")
    return cov_ts

def write_json(in_dict, filename):
    js = json.dumps(in_dict)
    f = open(f"/content/drive/My Drive/emgData/{filename}.json","w")
    f.write(js)
    f.close()
    f = open(f"/content/drive/My Drive/emgData/{filename}{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.json","w")
    f.write(js)
    f.close()

def runjob (subjects, mf_classifiers, ts_classifiers, intlength, stepsize, filename):
  subject_result = {subject:{} for subject in subjects}

  for i in subjects:
    print(f"participant {i} (intlength: {intlength}, stepsize: {stepsize})")
    emg, grasp, grasp_rep = load_data(i)  
    X,y = make_Xy(emg,grasp,grasp_rep,intlength,stepsize)
    cov = make_cov(X)
    for mfc in mf_classifiers:
      acc = classify(cov,y,mfc)
      subject_result[i] = {**subject_result[i], f"{mfc['classifier']} ({mfc['params']})": acc}
      print(subject_result)
      write_json(subject_result,filename)

    ts_proj = ts_projection(cov)
    for tsc in ts_classifiers:
      acc = classify(ts_proj,y,tsc)
      subject_result[i] = {**subject_result[i], f"{tsc['classifier'].__name__} ({tsc['params']})": acc}   
      print(subject_result)
      write_json(subject_result,filename)


In [5]:
subs = ["010","016","017","018"][::-1]
ts_classifiers = [{"classifier": tsvm.SVC, "params": {"kernel": "rbf", "gamma": 0.2}},
                  {"classifier": tsvm.SVC, "params": {"kernel": "rbf", "gamma": 0.3}},
                  {"classifier": tsvm.SVC, "params": {"kernel": "rbf", "gamma": 0.4}},
                  {"classifier": tsvm.SVC, "params": {"kernel": "rbf", "gamma": 0.5}},
                  {"classifier": tsvm.SVC, "params": {"kernel": "rbf", "gamma": 0.6}}]
mf_classifiers = []

#for step in [200,100,50,25]:
#  runjob(["010"], mf_classifiers, ts_classifiers, 400, step, "010step25-200.json")


runjob(subs, mf_classifiers, ts_classifiers, 400, 25, "010u016-018.json")


participant 018 (intlength: 400, stepsize: 25)
X,y sliding window time: 7.16s for (287160, 12, 400)
covariance estimation in 32.98 seconds.
tangent space projection in 574.54 seconds.
classify with SVC in 232.27 seconds. result: 0.99
{'018': {"SVC ({'kernel': 'rbf', 'gamma': 0.2})": 0.993453127176487}, '017': {}, '016': {}, '010': {}}
classify with SVC in 298.66 seconds. result: 1.00
{'018': {"SVC ({'kernel': 'rbf', 'gamma': 0.2})": 0.993453127176487, "SVC ({'kernel': 'rbf', 'gamma': 0.3})": 0.9955669313274829}, '017': {}, '016': {}, '010': {}}
classify with SVC in 373.24 seconds. result: 1.00
{'018': {"SVC ({'kernel': 'rbf', 'gamma': 0.2})": 0.993453127176487, "SVC ({'kernel': 'rbf', 'gamma': 0.3})": 0.9955669313274829, "SVC ({'kernel': 'rbf', 'gamma': 0.4})": 0.9951977991363699}, '017': {}, '016': {}, '010': {}}
classify with SVC in 459.90 seconds. result: 0.99
{'018': {"SVC ({'kernel': 'rbf', 'gamma': 0.2})": 0.993453127176487, "SVC ({'kernel': 'rbf', 'gamma': 0.3})": 0.995566931327

NameError: ignored

In [0]:

"""
TODO:
KNN classifier testen
alle anderen metriken testen
gucken, ob man gelernte svm vom einen auf anderen participant übertragen kann
"""



"""
accuracy for intlength: 400 and step: 400
cpu: 0.77s for (15778, 12, 400)
covariance estimation in 1.74 seconds.
tangent space projection in 27.96 seconds.
classify with SVC in 81.86 seconds.
estimated covariances in 1.72 seconds.
classify with MDM in 82.55 seconds.
SVC: 0.62
MDM: 0.51
accuracy for intlength: 400 and step: 200
cpu: 1.05s for (31551, 12, 400)
covariance estimation in 3.37 seconds.
tangent space projection in 53.15 seconds.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
classify with SVC in 153.97 seconds.
estimated covariances in 3.36 seconds.
classify with MDM in 174.65 seconds.
SVC: 0.67
MDM: 0.53
accuracy for intlength: 400 and step: 150
cpu: 1.22s for (47321, 12, 400)
covariance estimation in 5.15 seconds.
tangent space projection in 81.30 seconds.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
classify with SVC in 223.39 seconds.
estimated covariances in 5.04 seconds.
classify with MDM in 252.52 seconds.
SVC: 0.69
MDM: 0.54
accuracy for intlength: 400 and step: 100
cpu: 1.49s for (63095, 12, 400)
covariance estimation in 7.05 seconds.
tangent space projection in 109.59 seconds.
classify with SVC in 292.27 seconds.
estimated covariances in 6.86 seconds.
classify with MDM in 343.29 seconds.
SVC: 0.70
MDM: 0.54
accuracy for intlength: 400 and step: 50
cpu: 2.91s for (126185, 12, 400)
covariance estimation in 13.83 seconds.
tangent space projection in 226.27 seconds.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
classify with SVC in 570.11 seconds.
estimated covariances in 14.18 seconds.
classify with MDM in 682.42 seconds.
SVC: 0.71
MDM: 0.54
accuracy for intlength: 400 and step: 25
cpu: 8.37s for (252362, 12, 400)
covariance estimation in 27.85 seconds.
tangent space projection in 449.58 seconds.
classify with SVC in 2506.73 seconds.
estimated covariances in 27.55 seconds.
classify with MDM in 1353.05 seconds.
SVC: 0.71
MDM: 0.54
accuracy for intlength: 400 and step: 20
accuracy for intlength: 400 and step: 100
cpu: 1.70s for (63095, 12, 400)
covariance estimation in 6.92 seconds.
tangent space projection in 109.53 seconds.
classify with SVC in 45.15 seconds.
SVC(rbf): 0.81
accuracy for intlength: 400 and step: 50
cpu: 4.05s for (126185, 12, 400)
covariance estimation in 14.03 seconds.
tangent space projection in 217.51 seconds.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
classify with SVC in 131.87 seconds.
SVC(rbf): 0.84
accuracy for intlength: 400 and step: 25
cpu: 9.48s for (252362, 12, 400)
covariance estimation in 28.18 seconds.
classify with SVC in 642.65 seconds.
SVC(rbf): 0.87
accuracy for intlength: 400 and step: 20
cpu: 7.39s for (315451, 12, 400)
covariance estimation in 35.62 seconds.
tangent space projection in 580.25 seconds.
classify with SVC in 1199.34 seconds.
SVC: 0.87


accuracy participant 10 (intlength: 400, stepsize:25)
cpu: 5.38s for (252362, 12, 400)
covariance estimation in 28.30 seconds.
tangent space projection in 453.79 seconds.
classify with SVC in 641.79 seconds.
SVC (rbf): 0.87
accuracy participant 11 (intlength: 400, stepsize:25)
cpu: 7.12s for (287708, 12, 400)
covariance estimation in 33.05 seconds.
tangent space projection in 513.69 seconds.
classify with SVC in 1332.98 seconds.
SVC (rbf): 0.81
accuracy participant 12 (intlength: 400, stepsize:25)
cpu: 8.38s for (287069, 12, 400)
covariance estimation in 32.26 seconds.
tangent space projection in 517.04 seconds.
classify with SVC in 578.74 seconds.
SVC (rbf): 0.90
accuracy participant 13 (intlength: 400, stepsize:25)
cpu: 9.10s for (288535, 12, 400)
covariance estimation in 31.63 seconds.
tangent space projection in 520.53 seconds.
classify with SVC in 527.24 seconds.
SVC (rbf): 0.90
accuracy participant 14 (intlength: 400, stepsize:25)
cpu: 10.09s for (294391, 12, 400)
covariance estimation in 32.91 seconds.
tangent space projection in 585.14 seconds.
classify with SVC in 773.90 seconds.
SVC (rbf): 0.86
accuracy participant 15 (intlength: 400, stepsize:25)
cpu: 8.38s for (292671, 12, 400)
covariance estimation in 31.09 seconds.
tangent space projection in 568.02 seconds.
classify with SVC in 827.47 seconds.
SVC (rbf): 0.88
accuracy participant 16 (intlength: 400, stepsize:25)
cpu: 10.28s for (289000, 12, 400)
covariance estimation in 31.01 seconds.
tangent space projection in 575.57 seconds.
classify with SVC in 867.50 seconds.
SVC (rbf): 0.85
accuracy participant 17 (intlength: 400, stepsize:25)
cpu: 8.63s for (286972, 12, 400)
covariance estimation in 31.98 seconds.
tangent space projection in 536.49 seconds.
classify with SVC in 523.95 seconds.
SVC (rbf): 0.91
accuracy participant 18 (intlength: 400, stepsize:25)
cpu: 7.54s for (287160, 12, 400)
covariance estimation in 21.11 seconds.
tangent space projection in 406.38 seconds.
classify with SVC in 174.51 seconds.
SVC (rbf): 0.84
accuracy participant 19 (intlength: 400, stepsize:25)
cpu: 7.75s for (287011, 12, 400)
covariance estimation in 31.04 seconds.
tangent space projection in 560.11 seconds.
classify with SVC in 178.37 seconds.
SVC (rbf): 0.84
accuracy participant 20 (intlength: 400, stepsize:25)
cpu: 8.84s for (287393, 12, 400)
covariance estimation in 31.38 seconds.
tangent space projection in 500.87 seconds.
classify with SVC in 116.77 seconds.
SVC (rbf): 0.92



accuracy amputated participant 101 (intlength: 400, stepsize:25)
cpu: 8.72s for (279295, 12, 400)
covariance estimation in 29.43 seconds.
tangent space projection in 530.96 seconds.
classify with SVC in 141.67 seconds.
SVC (rbf): 0.86
accuracy amputated participant 102 (intlength: 400, stepsize:25)
cpu: 10.48s for (280329, 12, 400)
covariance estimation in 30.39 seconds.
tangent space projection in 616.35 seconds.
classify with SVC in 168.63 seconds.
SVC (rbf): 0.78
accuracy amputated participant 103 (intlength: 400, stepsize:25)
cpu: 8.51s for (281925, 12, 400)
covariance estimation in 30.73 seconds.
tangent space projection in 508.26 seconds.
classify with SVC in 126.08 seconds.
SVC (rbf): 0.87
accuracy amputated participant 104 (intlength: 400, stepsize:25)
cpu: 6.79s for (280240, 12, 400)
covariance estimation in 29.50 seconds.
tangent space projection in 488.79 seconds.
classify with SVC in 109.44 seconds.
SVC (rbf): 0.90
accuracy amputated participant 105 (intlength: 400, stepsize:25)
cpu: 8.65s for (281315, 12, 400)
covariance estimation in 29.64 seconds.
tangent space projection in 536.60 seconds.
classify with SVC in 144.16 seconds.
SVC (rbf): 0.85



accuracy participant 10 (intlength: 400, stepsize:25)
cpu: 12.07s for (252362, 12, 400)
covariance estimation in 27.59 seconds.
tangent space projection in 439.10 seconds.
classify with SVC in 166.85 seconds.
SVC (poly,deg: 3): 0.76
accuracy participant 11 (intlength: 400, stepsize:25)
cpu: 6.99s for (287708, 12, 400)
covariance estimation in 32.31 seconds.
tangent space projection in 500.84 seconds.
classify with SVC in 248.69 seconds.
SVC (poly,deg: 3): 0.74
accuracy participant 12 (intlength: 400, stepsize:25)
cpu: 6.27s for (287069, 12, 400)
covariance estimation in 32.12 seconds.
tangent space projection in 510.89 seconds.
classify with SVC in 170.53 seconds.
SVC (poly,deg: 3): 0.85
accuracy participant 13 (intlength: 400, stepsize:25)
cpu: 8.44s for (288535, 12, 400)
covariance estimation in 31.03 seconds.
tangent space projection in 502.05 seconds.
classify with SVC in 153.27 seconds.
SVC (poly,deg: 3): 0.84
accuracy participant 14 (intlength: 400, stepsize:25)
cpu: 5.24s for (294391, 12, 400)
covariance estimation in 32.15 seconds.
tangent space projection in 574.65 seconds.
classify with SVC in 189.80 seconds.
SVC (poly,deg: 3): 0.80

accuracy participant 10 (intlength: 400, stepsize:25)
cpu: 6.44s for (252362, 12, 400)
covariance estimation in 27.67 seconds.
tangent space projection in 447.19 seconds.
classify with SVC in 188.02 seconds.
SVC (poly,deg: 5): 0.65
accuracy participant 11 (intlength: 400, stepsize:25)
cpu: 9.32s for (287708, 12, 400)
covariance estimation in 30.91 seconds.
tangent space projection in 507.52 seconds.
classify with SVC in 251.39 seconds.
SVC (poly,deg: 5): 0.67
"""
print()


In [0]:
emg, grasp, grasp_rep = load_data("010")  

In [0]:
start = time.time()
X,y = sliding_windows_X(emg,grasp,grasp_rep,intlength=400,step_size=200)
end = time.time()
print(f"cpu: {(end-start):.2f}s for {X.shape}")
print(X.shape)

cpu: 1.21s for (31551, 12, 400)
(31551, 12, 400)


In [0]:
for i in [20000,40000,80000]:
  X1 = X[:i]
  y1 = y[:i]
  a1 = classify_SVC(X1,y1,SVC).mean()
  print(a1)
  #0.54 für 400,20

covariance estimation in 1.44 seconds.
tangent space projection in 25.89 seconds.
classify with SVC in 24.15 seconds.
0.9387000000000001
covariance estimation in 2.79 seconds.
tangent space projection in 51.58 seconds.


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


classify with SVC in 258.34 seconds.
0.8252
covariance estimation in 5.35 seconds.
tangent space projection in 93.29 seconds.


KeyboardInterrupt: ignored

In [0]:
a1

array([0.73375594, 0.76349206, 0.73650794, 0.74126984, 0.58888889])

In [0]:
X1 = X[:100000]
y1 = y[:100000]

startcov = time.time()
cov = pr.estimation.Covariances().fit_transform(X1)
endcov = time.time()
print(f"covariance estimation in {(endcov - startcov):.2f} seconds.")
startcov = time.time()

ts = TangentSpace()
cov_ts = ts.fit_transform(cov)
endcov = time.time()
print(f"tangent space projection in {(endcov - startcov):.2f} seconds.")

startcov = time.time()
svc = SVC(kernel='linear')
res = svc.fit(cov_ts,y1)
#accuracy = cross_val_score(svc, cov_ts, y1, n_jobs=2)
endcov = time.time()
print(f"classify with SVC in {(endcov - startcov):.2f} seconds.")

startcov = time.time()
svc = tsvm.SVC(kernel='linear')
res = svc.fit(cov_ts,y1)

#accuracy = cross_val_score(svc, cov_ts, y1, n_jobs=2)
endcov = time.time()
print(f"classify with SVC in {(endcov - startcov):.2f} seconds.")

covariance estimation in 2.20 seconds.
tangent space projection in 42.08 seconds.
classify with SVC in 63.34 seconds.


KeyboardInterrupt: ignored